#### emulation of Linux tree command in Python ####

In [1]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

In [12]:
list_files('/Users/johngage/Documents/blog/content/notebooks')

notebooks/
    lab05WorldOriginal.ipynb
    lab05WorldComplete.ipynb
    Untitled.ipynb
    Batteries.ipynb
    lab05Song1.ipynb
    lab05WorldComplete-Copy1.ipynb
    lab05Song2.ipynb
    .ipynb_checkpoints/
        lab05Song1-checkpoint.ipynb
        lab05WorldOriginal-checkpoint.ipynb
        lab05WorldComplete-checkpoint.ipynb
        lab05Song2-checkpoint.ipynb
        Untitled-checkpoint.ipynb
        Batteries-checkpoint.ipynb
        lab05WorldComplete-Copy1-checkpoint.ipynb


In [20]:
!pwd


/Users/johngage/Documents/blog/content/notebooks


In [25]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))


In [30]:
paths = DisplayablePath.make_tree(Path('/Users/johngage/Documents/blog/content/'))

In [31]:
for path in paths:
    print(path.displayable())


content/
├── articles/
│   ├── Battery.md
│   ├── CSWBHackathon3.md
│   ├── CWBHackathon1.md
│   ├── CWBHackathon2.md
│   ├── DataScienceWorld.md
│   ├── DKMarin.md
│   ├── first.rst
│   ├── HowThisSiteWasCreated.md
│   ├── LittleTable.html
│   ├── Mt.Mantap Nuclear Test.md
│   ├── NewRST.rst
│   ├── TahoeBuoy.md
│   ├── TeamsOnSlack.md
│   ├── Tues.md
│   ├── Tues.rst
│   ├── VotingSep27.md
│   ├── waterhackathon-followup.md
│   └── Waveglider.md
├── Draft/
├── extra/
│   ├── custom.css
│   └── custom.js
├── images/
│   ├── .DS_Store
│   └── sunlight.png
├── notebooks/
│   ├── .ipynb_checkpoints/
│   │   ├── Batteries-checkpoint.ipynb
│   │   ├── lab05Song1-checkpoint.ipynb
│   │   ├── lab05Song2-checkpoint.ipynb
│   │   ├── lab05WorldComplete-checkpoint.ipynb
│   │   ├── lab05WorldComplete-Copy1-checkpoint.ipynb
│   │   ├── lab05WorldOriginal-checkpoint.ipynb
│   │   └── Untitled-checkpoint.ipynb
│   ├── Batteries.ipynb
│   ├── lab05Song1.ipynb
│   ├── lab05Song2.ipynb
│   ├── lab05W